# App Rating Predictor
## Building an app rating predictor using data scraped from the android market

Import packages

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
pd.set_option("display.max_columns", None) #Show all columns, or else some will just be "..."

In [2]:
df = pd.read_csv("./datasets/google_app_scrap.csv")
df.head()

,Unnamed: 0,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,CURRENCY,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,FAMILY_GENRE,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,RELEASED_DATE,LAST_UPDATED,DAY_SCRAPED
0,0,Skincare Routine,4.510000,Beauty,topselling_paid,609.0,34,11,23,65,465,89.0,"10,000+",10000,17616,False,3.99,USD,16M,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,Everyone,NaN,False,False,False,"Aug 13, 2018",14/2/22,8/3/22
1,1,Booksy Biz: For Businesses,4.313544,Beauty,topgrossing,13176.0,1587,365,464,659,10095,1920.0,"1,000,000+",1000000,1270756,True,0.00,USD,24M,Booksy International sp. z o.o.,"ul. Łucka 2/4/6, lok. U 4,\n00-845 Warsaw\nPoland",Teen,NaN,False,True,False,"May 22, 2014",3/3/22,8/3/22
2,2,Perfect365 Makeup Photo Editor,4.295216,Beauty,topgrossing,782877.0,62917,22800,55202,121258,520690,49923.0,"50,000,000+",50000000,54737806,True,0.00,USD,201M,"Perfect365, Inc.","Perfect365, Inc.\n101 Jefferson Drive\nMenlo P...",Everyone,NaN,True,True,False,"Feb 19, 2013",4/3/22,8/3/22
3,3,Glowbe – Face Yoga & Exercise,3.136364,Beauty,topgrossing,95.0,38,6,0,2,47,29.0,"10,000+",10000,29466,True,0.00,USD,67M,Glowbe LTD,"4103, Limassol Cyprus",Everyone,NaN,False,True,False,"Dec 2, 2020",20/2/22,8/3/22
4,4,PhiApp,4.233333,Beauty,topgrossing,916.0,122,15,61,45,671,26.0,"100,000+",100000,302978,True,0.00,USD,Varies with device,PhiAcademy GmbH,"Gartengasse 8/8\n1050, Vienna\nAustria",Everyone,NaN,False,True,False,"Mar 13, 2015",20/1/22,8/3/22


<a id = 'data cleaning'></a>
# Data Cleaning

In [3]:
df.describe(include="all")

,Unnamed: 0,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,CURRENCY,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,FAMILY_GENRE,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,RELEASED_DATE,LAST_UPDATED,DAY_SCRAPED
count,23952.00000,23952,23934.000000,23952,23952,2.393400e+04,2.395200e+04,2.395200e+04,2.395200e+04,2.395200e+04,2.395200e+04,2.393400e+04,23952,2.395200e+04,2.395200e+04,23952,23952.000000,23952,23952,23952,19420,23952,0.0,23952,23952,23952,23762,23952,23952
unique,NaN,18496,NaN,48,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22,NaN,NaN,2,NaN,1,666,12343,8614,5,NaN,2,2,2,3935,1600,1
top,NaN,Solitaire,NaN,Sports,topselling_free,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,000,000+",NaN,NaN,True,NaN,USD,Varies with device,My Town Games Ltd,"Ha-Rekhev St 1, Tel Aviv-Yafo",Everyone,NaN,False,True,False,"Feb 21, 2022",4/3/22,8/3/22
freq,NaN,27,NaN,944,10593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4427,NaN,NaN,18520,NaN,23952,3170,137,137,17530,NaN,12182,14509,23439,44,955,23952
mean,11975.50000,NaN,3.988836,NaN,NaN,3.899129e+05,4.401561e+04,9.365627e+03,1.810578e+04,4.228248e+04,2.758255e+05,1.778189e+04,NaN,1.891777e+07,3.073138e+07,NaN,1.422892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,6914.49116,NaN,1.044631,NaN,NaN,3.676933e+06,7.961011e+05,1.007322e+05,1.726730e+05,3.569045e+05,2.428996e+06,1.179491e+05,NaN,2.201441e+08,2.802337e+08,NaN,8.832669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.00000,NaN,0.000000,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,5987.75000,NaN,3.891892,NaN,NaN,6.412500e+02,5.800000e+01,1.200000e+01,2.300000e+01,5.100000e+01,3.927500e+02,6.400000e+01,NaN,5.000000e+04,5.644300e+04,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,11975.50000,NaN,4.300697,NaN,NaN,8.450000e+03,6.805000e+02,1.900000e+02,3.470000e+02,7.885000e+02,5.698000e+03,5.750000e+02,NaN,5.000000e+05,8.288390e+05,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,17963.25000,NaN,4.561213,NaN,NaN,7.308575e+04,5.809500e+03,1.719250e+03,3.201000e+03,7.650000e+03,5.187900e+04,4.237750e+03,NaN,5.000000e+06,7.747612e+06,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23952 entries, 0 to 23951
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         23952 non-null  int64  
 1   APP_NAME           23952 non-null  object 
 2   RATING             23934 non-null  float64
 3   CATEGORY           23952 non-null  object 
 4   COLLECTIONS        23952 non-null  object 
 5   RATING_COUNT       23934 non-null  float64
 6   1_STAR_RATINGS     23952 non-null  int64  
 7   2_STAR_RATINGS     23952 non-null  int64  
 8   3_STAR_RATINGS     23952 non-null  int64  
 9   4_STAR_RATINGS     23952 non-null  int64  
 10  5_STAR_RATINGS     23952 non-null  int64  
 11  REVIEW_COUNT       23934 non-null  float64
 12  INSTALLS           23952 non-null  object 
 13  MIN_INSTALLS       23952 non-null  int64  
 14  MAX_INSTALLS       23952 non-null  int64  
 15  FREE               23952 non-null  bool   
 16  PRICE              239

Some initial observations:
1. UNNAMED: 0 column is same as the index
2. There are no values in FAMILY_GENRE
3. All the values in CURRENCY are the same (usd) which indicates the currency of the price of the app
4. The 5 n_STAR_RATING columns does not always sum up to RATING_COUNT
5. SIZE contains string objects with postfix of K, M, G to indicate the size of the app in bytes
6. The formats for columns with date objects are inconsistent

### Remove useless columns that contain same/empty values which won't contribute to our EDA
We will remove:
- `FAMILY_GENRE`
- `Unnamed: 0`
- `CURRENCY`

In [5]:
df = df.drop(columns=['Unnamed: 0', 'FAMILY_GENRE', 'CURRENCY'])
df.columns

Index(['APP_NAME', 'RATING', 'CATEGORY', 'COLLECTIONS', 'RATING_COUNT',
       '1_STAR_RATINGS', '2_STAR_RATINGS', '3_STAR_RATINGS', '4_STAR_RATINGS',
       '5_STAR_RATINGS', 'REVIEW_COUNT', 'INSTALLS', 'MIN_INSTALLS',
       'MAX_INSTALLS', 'FREE', 'PRICE', 'SIZE', 'DEVELOPER',
       'DEVELOPER_ADDRESS', 'CONTENT_RATING', 'AD_SUPPORTED',
       'IN_APP_PURCHASES', 'EDITORS_CHOICE', 'RELEASED_DATE', 'LAST_UPDATED',
       'DAY_SCRAPED'],
      dtype='object')

### Tidy up `RATING_COUNT`
RATING_COUNT is inconsistent. So we will replace the values in `RATING_COUNT` with the sum of `1_STAR_RATING`, `2_STAR_RATING`, `3_STAR_RATING`, `4_STAR_RATING`, `5_STAR_RATING`. We also drop the apps with 0 rating count (the app is too new)

In [6]:
df['RATING_COUNT'] = df[['1_STAR_RATINGS', '2_STAR_RATINGS', '3_STAR_RATINGS', '4_STAR_RATINGS', '5_STAR_RATINGS']].sum(axis=1)
df = df[df["RATING_COUNT"] != 0]
df.reset_index(inplace=True, drop=True)
df.head()

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,RELEASED_DATE,LAST_UPDATED,DAY_SCRAPED
0,Skincare Routine,4.510000,Beauty,topselling_paid,598,34,11,23,65,465,89.0,"10,000+",10000,17616,False,3.99,16M,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,Everyone,False,False,False,"Aug 13, 2018",14/2/22,8/3/22
1,Booksy Biz: For Businesses,4.313544,Beauty,topgrossing,13170,1587,365,464,659,10095,1920.0,"1,000,000+",1000000,1270756,True,0.00,24M,Booksy International sp. z o.o.,"ul. Łucka 2/4/6, lok. U 4,\n00-845 Warsaw\nPoland",Teen,False,True,False,"May 22, 2014",3/3/22,8/3/22
2,Perfect365 Makeup Photo Editor,4.295216,Beauty,topgrossing,782867,62917,22800,55202,121258,520690,49923.0,"50,000,000+",50000000,54737806,True,0.00,201M,"Perfect365, Inc.","Perfect365, Inc.\n101 Jefferson Drive\nMenlo P...",Everyone,True,True,False,"Feb 19, 2013",4/3/22,8/3/22
3,Glowbe – Face Yoga & Exercise,3.136364,Beauty,topgrossing,93,38,6,0,2,47,29.0,"10,000+",10000,29466,True,0.00,67M,Glowbe LTD,"4103, Limassol Cyprus",Everyone,False,True,False,"Dec 2, 2020",20/2/22,8/3/22
4,PhiApp,4.233333,Beauty,topgrossing,914,122,15,61,45,671,26.0,"100,000+",100000,302978,True,0.00,Varies with device,PhiAcademy GmbH,"Gartengasse 8/8\n1050, Vienna\nAustria",Everyone,False,True,False,"Mar 13, 2015",20/1/22,8/3/22


### Clean `SIZE`
`SIZE` are string objects, we will convert them into integers for integer operations in EDA. <br>
For example: 12M -> 12000000, 12K -> 12000, varies with device -> NaN (handle this later)

In [7]:
df["SIZE"] = df["SIZE"].str.replace("Varies with device","-1") #replacing "Varies with device" with -1 
df["SIZE"] = df["SIZE"].str.replace(",","") #replacing ',' with an empty string

#changing k to (10**3), M to (10**6) and G to (10**9)
df["SIZE"] = (df["SIZE"].replace(r'[kMG]+$', '', regex=True).astype(float) * \
                   df["SIZE"].str.extract(r'[\d\.]+([kMG]+)', expand=False)
                   .fillna(1)
                   .replace(['k','M','G'], [10**3, 10**6,10**9]).astype(int))
df['SIZE'] = df['SIZE'].astype('int64') #converting float to int64
df.loc[df['SIZE'] == -1, 'SIZE'] = np.nan # convert -1 to NaN
df.head()

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,RELEASED_DATE,LAST_UPDATED,DAY_SCRAPED
0,Skincare Routine,4.510000,Beauty,topselling_paid,598,34,11,23,65,465,89.0,"10,000+",10000,17616,False,3.99,16000000.0,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,Everyone,False,False,False,"Aug 13, 2018",14/2/22,8/3/22
1,Booksy Biz: For Businesses,4.313544,Beauty,topgrossing,13170,1587,365,464,659,10095,1920.0,"1,000,000+",1000000,1270756,True,0.00,24000000.0,Booksy International sp. z o.o.,"ul. Łucka 2/4/6, lok. U 4,\n00-845 Warsaw\nPoland",Teen,False,True,False,"May 22, 2014",3/3/22,8/3/22
2,Perfect365 Makeup Photo Editor,4.295216,Beauty,topgrossing,782867,62917,22800,55202,121258,520690,49923.0,"50,000,000+",50000000,54737806,True,0.00,201000000.0,"Perfect365, Inc.","Perfect365, Inc.\n101 Jefferson Drive\nMenlo P...",Everyone,True,True,False,"Feb 19, 2013",4/3/22,8/3/22
3,Glowbe – Face Yoga & Exercise,3.136364,Beauty,topgrossing,93,38,6,0,2,47,29.0,"10,000+",10000,29466,True,0.00,67000000.0,Glowbe LTD,"4103, Limassol Cyprus",Everyone,False,True,False,"Dec 2, 2020",20/2/22,8/3/22
4,PhiApp,4.233333,Beauty,topgrossing,914,122,15,61,45,671,26.0,"100,000+",100000,302978,True,0.00,NaN,PhiAcademy GmbH,"Gartengasse 8/8\n1050, Vienna\nAustria",Everyone,False,True,False,"Mar 13, 2015",20/1/22,8/3/22


### Reformat dates

`RELEASED_DATE`, `LAST_UPDATED`, `DAY_SCRAPED` are string objects, and their format are inconsistent and different. Hence, we will reformat them into **"dd/mm/yyyy"**.

In [8]:
#Convert strings to datetime
df['RELEASED_DATE'] = pd.to_datetime(df['RELEASED_DATE'])
df['LAST_UPDATED'] = pd.to_datetime(df['LAST_UPDATED'], format="%d/%m/%y")
df['DAY_SCRAPED'] = pd.to_datetime(df['DAY_SCRAPED'], format="%d/%m/%y")

#reformat date
df['RELEASED_DATE'] = df['RELEASED_DATE'].dt.strftime('%d/%m/%Y')
df['LAST_UPDATED'] = df['LAST_UPDATED'].dt.strftime('%d/%m/%Y')
df['DAY_SCRAPED'] = df['DAY_SCRAPED'].dt.strftime('%d/%m/%Y')
df.head()

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,RELEASED_DATE,LAST_UPDATED,DAY_SCRAPED
0,Skincare Routine,4.510000,Beauty,topselling_paid,598,34,11,23,65,465,89.0,"10,000+",10000,17616,False,3.99,16000000.0,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,Everyone,False,False,False,13/08/2018,14/02/2022,08/03/2022
1,Booksy Biz: For Businesses,4.313544,Beauty,topgrossing,13170,1587,365,464,659,10095,1920.0,"1,000,000+",1000000,1270756,True,0.00,24000000.0,Booksy International sp. z o.o.,"ul. Łucka 2/4/6, lok. U 4,\n00-845 Warsaw\nPoland",Teen,False,True,False,22/05/2014,03/03/2022,08/03/2022
2,Perfect365 Makeup Photo Editor,4.295216,Beauty,topgrossing,782867,62917,22800,55202,121258,520690,49923.0,"50,000,000+",50000000,54737806,True,0.00,201000000.0,"Perfect365, Inc.","Perfect365, Inc.\n101 Jefferson Drive\nMenlo P...",Everyone,True,True,False,19/02/2013,04/03/2022,08/03/2022
3,Glowbe – Face Yoga & Exercise,3.136364,Beauty,topgrossing,93,38,6,0,2,47,29.0,"10,000+",10000,29466,True,0.00,67000000.0,Glowbe LTD,"4103, Limassol Cyprus",Everyone,False,True,False,02/12/2020,20/02/2022,08/03/2022
4,PhiApp,4.233333,Beauty,topgrossing,914,122,15,61,45,671,26.0,"100,000+",100000,302978,True,0.00,NaN,PhiAcademy GmbH,"Gartengasse 8/8\n1050, Vienna\nAustria",Everyone,False,True,False,13/03/2015,20/01/2022,08/03/2022


### Dealing with NULL values

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22824 entries, 0 to 22823
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   APP_NAME           22824 non-null  object 
 1   RATING             22824 non-null  float64
 2   CATEGORY           22824 non-null  object 
 3   COLLECTIONS        22824 non-null  object 
 4   RATING_COUNT       22824 non-null  int64  
 5   1_STAR_RATINGS     22824 non-null  int64  
 6   2_STAR_RATINGS     22824 non-null  int64  
 7   3_STAR_RATINGS     22824 non-null  int64  
 8   4_STAR_RATINGS     22824 non-null  int64  
 9   5_STAR_RATINGS     22824 non-null  int64  
 10  REVIEW_COUNT       22824 non-null  float64
 11  INSTALLS           22824 non-null  object 
 12  MIN_INSTALLS       22824 non-null  int64  
 13  MAX_INSTALLS       22824 non-null  int64  
 14  FREE               22824 non-null  bool   
 15  PRICE              22824 non-null  float64
 16  SIZE               197

By observation and previous cleaning steps, there are 6 columns with NULL values which are:
- REVIEW_COUNT
- RATING
- RATING_COUNT
- RELEASED_DATE
- SIZE
- DEVELOPER_ADDRESS 

For `REVIEW_COUNT`, `RATING`, `RATING_COUNT`, the values are null value most likely because the app is too new and no customers have done any review on them, so **remove rows with NULL column values** from the dataset.

In [10]:
df = df.dropna(axis=0, subset=['REVIEW_COUNT', 'RATING', 'RATING_COUNT'])
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22824 entries, 0 to 22823
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   APP_NAME           22824 non-null  object 
 1   RATING             22824 non-null  float64
 2   CATEGORY           22824 non-null  object 
 3   COLLECTIONS        22824 non-null  object 
 4   RATING_COUNT       22824 non-null  int64  
 5   1_STAR_RATINGS     22824 non-null  int64  
 6   2_STAR_RATINGS     22824 non-null  int64  
 7   3_STAR_RATINGS     22824 non-null  int64  
 8   4_STAR_RATINGS     22824 non-null  int64  
 9   5_STAR_RATINGS     22824 non-null  int64  
 10  REVIEW_COUNT       22824 non-null  float64
 11  INSTALLS           22824 non-null  object 
 12  MIN_INSTALLS       22824 non-null  int64  
 13  MAX_INSTALLS       22824 non-null  int64  
 14  FREE               22824 non-null  bool   
 15  PRICE              22824 non-null  float64
 16  SIZE               197

As for `RELEASED_DATE`, fill the **null values with the median**.

But first we create to new columns to replace `RELEASED_DATE` and `LAST_UPDATED` to contain numerical values instead of Date object.

In [11]:
for i in range(df.shape[0]):
    if (pd.isnull(df.loc[i, "LAST_UPDATED"])):
        df.loc[i, "DAYS_SINCE_UPDATE"] = np.nan
        continue
    date_from = pd.to_datetime(df.loc[i, "LAST_UPDATED"], format="%d/%m/%Y").date()
    date_till = pd.to_datetime(df.loc[i, "DAY_SCRAPED"], format="%d/%m/%Y").date()
    date_elasped = np.int64((date_till-date_from).days)
    df.loc[i, "DAYS_SINCE_UPDATE"] = date_elasped
    
    if (pd.isnull(df.loc[i, "RELEASED_DATE"])):
        df.loc[i, "DAYS_SINCE_RELEASED"] = np.nan
        continue
    
    date_from = pd.to_datetime(df.loc[i, "RELEASED_DATE"], format="%d/%m/%Y").date()
    date_till = pd.to_datetime(df.loc[i, "DAY_SCRAPED"], format="%d/%m/%Y").date()
    date_elasped = np.int64((date_till-date_from).days)
    df.loc[i, "DAYS_SINCE_RELEASED"] = date_elasped
    
df.head()

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MIN_INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,RELEASED_DATE,LAST_UPDATED,DAY_SCRAPED,DAYS_SINCE_UPDATE,DAYS_SINCE_RELEASED
0,Skincare Routine,4.510000,Beauty,topselling_paid,598,34,11,23,65,465,89.0,"10,000+",10000,17616,False,3.99,16000000.0,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,Everyone,False,False,False,13/08/2018,14/02/2022,08/03/2022,22.0,1303.0
1,Booksy Biz: For Businesses,4.313544,Beauty,topgrossing,13170,1587,365,464,659,10095,1920.0,"1,000,000+",1000000,1270756,True,0.00,24000000.0,Booksy International sp. z o.o.,"ul. Łucka 2/4/6, lok. U 4,\n00-845 Warsaw\nPoland",Teen,False,True,False,22/05/2014,03/03/2022,08/03/2022,5.0,2847.0
2,Perfect365 Makeup Photo Editor,4.295216,Beauty,topgrossing,782867,62917,22800,55202,121258,520690,49923.0,"50,000,000+",50000000,54737806,True,0.00,201000000.0,"Perfect365, Inc.","Perfect365, Inc.\n101 Jefferson Drive\nMenlo P...",Everyone,True,True,False,19/02/2013,04/03/2022,08/03/2022,4.0,3304.0
3,Glowbe – Face Yoga & Exercise,3.136364,Beauty,topgrossing,93,38,6,0,2,47,29.0,"10,000+",10000,29466,True,0.00,67000000.0,Glowbe LTD,"4103, Limassol Cyprus",Everyone,False,True,False,02/12/2020,20/02/2022,08/03/2022,16.0,461.0
4,PhiApp,4.233333,Beauty,topgrossing,914,122,15,61,45,671,26.0,"100,000+",100000,302978,True,0.00,NaN,PhiAcademy GmbH,"Gartengasse 8/8\n1050, Vienna\nAustria",Everyone,False,True,False,13/03/2015,20/01/2022,08/03/2022,47.0,2552.0


Filling na values of `DAYS_SINCE_RELEASED`, `DAYS_SINCE_UPDATE`, `SIZE` with its median

In [12]:
median_dsr = df["DAYS_SINCE_RELEASED"].median()
median_dsu = df["DAYS_SINCE_UPDATE"].median()
median_size = df["SIZE"].median()
df["DAYS_SINCE_RELEASED"].fillna(median_dsr, inplace=True)
df["DAYS_SINCE_UPDATE"].fillna(median_dsu, inplace=True)
df["SIZE"].fillna(median_size, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22824 entries, 0 to 22823
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   APP_NAME             22824 non-null  object 
 1   RATING               22824 non-null  float64
 2   CATEGORY             22824 non-null  object 
 3   COLLECTIONS          22824 non-null  object 
 4   RATING_COUNT         22824 non-null  int64  
 5   1_STAR_RATINGS       22824 non-null  int64  
 6   2_STAR_RATINGS       22824 non-null  int64  
 7   3_STAR_RATINGS       22824 non-null  int64  
 8   4_STAR_RATINGS       22824 non-null  int64  
 9   5_STAR_RATINGS       22824 non-null  int64  
 10  REVIEW_COUNT         22824 non-null  float64
 11  INSTALLS             22824 non-null  object 
 12  MIN_INSTALLS         22824 non-null  int64  
 13  MAX_INSTALLS         22824 non-null  int64  
 14  FREE                 22824 non-null  bool   
 15  PRICE                22824 non-null 

Drop `RELEASED_DATE`, `LAST_UPDATED`, `DAY_SCRAPED`, `COLLECTIONS`, and `MIN_INSTALLS` because we dont need them anymore

In [13]:
df = df.drop(columns=['RELEASED_DATE', 'LAST_UPDATED', 'DAY_SCRAPED', 'COLLECTIONS', 'MIN_INSTALLS'])
df.columns

Index(['APP_NAME', 'RATING', 'CATEGORY', 'RATING_COUNT', '1_STAR_RATINGS',
       '2_STAR_RATINGS', '3_STAR_RATINGS', '4_STAR_RATINGS', '5_STAR_RATINGS',
       'REVIEW_COUNT', 'INSTALLS', 'MAX_INSTALLS', 'FREE', 'PRICE', 'SIZE',
       'DEVELOPER', 'DEVELOPER_ADDRESS', 'CONTENT_RATING', 'AD_SUPPORTED',
       'IN_APP_PURCHASES', 'EDITORS_CHOICE', 'DAYS_SINCE_UPDATE',
       'DAYS_SINCE_RELEASED'],
      dtype='object')

### Dealing with Addresses

The raw address of developers might be hard to use for eda, we will generate a new feature `COUNTRY` that holds the country the developer originated.

First, import the dataset of predicted country of each developer address that we generated using **Google Maps API**.

In [14]:
country_df = pd.read_csv("./datasets/developer_country.csv")
country_df.head()

,index,DEVELOPER_ADDRESS,COUNTRY
0,0,23 Stoneleigh Avenue\r\nBN1 8NP,United Kingdom
1,1,"ul. Łucka 2/4/6, lok. U 4,\r\n00-845 Warsaw\r\...",Poland
2,2,"Perfect365, Inc.\r\n101 Jefferson Drive\r\nMen...",Usa
3,3,"4103, Limassol Cyprus",Cyprus
4,4,"Gartengasse 8/8\r\n1050, Vienna\r\nAustria",Austria


Remove # from each row in `DEVELOPER_ADDRESS` column to match the format of mapping in the country_df

In [15]:
for i in range(df.shape[0]):
    developer_addr = df.loc[i, "DEVELOPER_ADDRESS"]
    if (pd.isnull(developer_addr)):
        continue
    df.loc[i, "DEVELOPER_ADDRESS"] = developer_addr.replace("#", " ")

# Map to country
df['COUNTRY']=df[['DEVELOPER_ADDRESS']].merge(country_df,how='left').COUNTRY
df.head()

,APP_NAME,RATING,CATEGORY,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,DAYS_SINCE_UPDATE,DAYS_SINCE_RELEASED,COUNTRY
0,Skincare Routine,4.510000,Beauty,598,34,11,23,65,465,89.0,"10,000+",17616,False,3.99,16000000.0,Mento Apps Ltd,23 Stoneleigh Avenue\nBN1 8NP,Everyone,False,False,False,22.0,1303.0,NaN
1,Booksy Biz: For Businesses,4.313544,Beauty,13170,1587,365,464,659,10095,1920.0,"1,000,000+",1270756,True,0.00,24000000.0,Booksy International sp. z o.o.,"ul. Łucka 2/4/6, lok. U 4,\n00-845 Warsaw\nPoland",Teen,False,True,False,5.0,2847.0,NaN
2,Perfect365 Makeup Photo Editor,4.295216,Beauty,782867,62917,22800,55202,121258,520690,49923.0,"50,000,000+",54737806,True,0.00,201000000.0,"Perfect365, Inc.","Perfect365, Inc.\n101 Jefferson Drive\nMenlo P...",Everyone,True,True,False,4.0,3304.0,NaN
3,Glowbe – Face Yoga & Exercise,3.136364,Beauty,93,38,6,0,2,47,29.0,"10,000+",29466,True,0.00,67000000.0,Glowbe LTD,"4103, Limassol Cyprus",Everyone,False,True,False,16.0,461.0,Cyprus
4,PhiApp,4.233333,Beauty,914,122,15,61,45,671,26.0,"100,000+",302978,True,0.00,47000000.0,PhiAcademy GmbH,"Gartengasse 8/8\n1050, Vienna\nAustria",Everyone,False,True,False,47.0,2552.0,NaN


Reposition the `COUNTRY` column

In [16]:
df.insert(17, 'COUNTRY', df.pop('COUNTRY'))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22824 entries, 0 to 22823
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   APP_NAME             22824 non-null  object 
 1   RATING               22824 non-null  float64
 2   CATEGORY             22824 non-null  object 
 3   RATING_COUNT         22824 non-null  int64  
 4   1_STAR_RATINGS       22824 non-null  int64  
 5   2_STAR_RATINGS       22824 non-null  int64  
 6   3_STAR_RATINGS       22824 non-null  int64  
 7   4_STAR_RATINGS       22824 non-null  int64  
 8   5_STAR_RATINGS       22824 non-null  int64  
 9   REVIEW_COUNT         22824 non-null  float64
 10  INSTALLS             22824 non-null  object 
 11  MAX_INSTALLS         22824 non-null  int64  
 12  FREE                 22824 non-null  bool   
 13  PRICE                22824 non-null  float64
 14  SIZE                 22824 non-null  float64
 15  DEVELOPER            22824 non-null 

There are 6510 apps with predicted developer country, which means more than half the apps do not have predicted country/registered country.

We will only use this info for eda only

### Sort the dataset by the `APP_NAME`

In [17]:
df.sort_values("APP_NAME", inplace = True)
df.reset_index(drop=True, inplace=True)
df.head()

,APP_NAME,RATING,CATEGORY,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,COUNTRY,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,DAYS_SINCE_UPDATE,DAYS_SINCE_RELEASED
0,"""Ghost Voice Catcher"" AUTO EVP",3.933333,Lifestyle,68,12,1,6,7,42,24.0,"1,000+",3608,False,5.99,1100000.0,BIG BEARD Studios,114 1/2 Ash st. Nashua NH 03060,Usa,Everyone,False,False,False,1820.0,3482.0
1,"""OXXO""",4.740000,Puzzle,975,23,0,15,94,843,54.0,"10,000+",21680,False,0.99,81000000.0,Hamster On Coke Games,ul Jugosłowiańska 37F/10\n73-110 Stargard Szcz...,NaN,Everyone,False,False,False,0.0,945.0
2,#DRIVE,4.434152,Racing,229679,9352,4740,15249,47804,152534,6473.0,"5,000,000+",9119244,True,0.00,132000000.0,Pixel Perfect Dude S.A.,ul. Polna 28/6 lok. 4\n62-800 Kalisz\nPolska,NaN,Everyone,True,True,False,4.0,1084.0
3,#DRIVE,4.434152,Racing,229679,9352,4740,15249,47804,152534,6473.0,"5,000,000+",9119244,True,0.00,132000000.0,Pixel Perfect Dude S.A.,ul. Polna 28/6 lok. 4\n62-800 Kalisz\nPolska,NaN,Everyone,True,True,False,4.0,1084.0
4,#SelfCare,4.463476,Role Playing,14932,865,336,1088,1353,11290,1501.0,"1,000,000+",1209692,True,0.00,83000000.0,TRU LUV Inc.,"186-720 King Street West\nToronto, Ontario, Ca...",NaN,Teen,False,True,False,952.0,1275.0


### Drop duplicated apps that has different categories, but turn their `CATEGORY` column into a String(array) 

In [18]:
temp_df = df.groupby("APP_NAME")["CATEGORY"].apply(list)
df.drop_duplicates(subset ="APP_NAME", keep = "first", inplace = True)
df.reset_index(drop=True, inplace=True)
i = 0
for tags in temp_df:
    to_add = []
    [to_add.append(x) for x in tags if x not in to_add]
    df.loc[i, "CATEGORY"] = str(to_add)
    i+=1
df.head() 

,APP_NAME,RATING,CATEGORY,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,COUNTRY,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,DAYS_SINCE_UPDATE,DAYS_SINCE_RELEASED
0,"""Ghost Voice Catcher"" AUTO EVP",3.933333,['Lifestyle'],68,12,1,6,7,42,24.0,"1,000+",3608,False,5.99,1100000.0,BIG BEARD Studios,114 1/2 Ash st. Nashua NH 03060,Usa,Everyone,False,False,False,1820.0,3482.0
1,"""OXXO""",4.740000,['Puzzle'],975,23,0,15,94,843,54.0,"10,000+",21680,False,0.99,81000000.0,Hamster On Coke Games,ul Jugosłowiańska 37F/10\n73-110 Stargard Szcz...,NaN,Everyone,False,False,False,0.0,945.0
2,#DRIVE,4.434152,['Racing'],229679,9352,4740,15249,47804,152534,6473.0,"5,000,000+",9119244,True,0.00,132000000.0,Pixel Perfect Dude S.A.,ul. Polna 28/6 lok. 4\n62-800 Kalisz\nPolska,NaN,Everyone,True,True,False,4.0,1084.0
3,#SelfCare,4.463476,['Role Playing'],14932,865,336,1088,1353,11290,1501.0,"1,000,000+",1209692,True,0.00,83000000.0,TRU LUV Inc.,"186-720 King Street West\nToronto, Ontario, Ca...",NaN,Teen,False,True,False,952.0,1275.0
4,#open Polyamorous + ENM Dating,2.550000,['Dating'],708,263,163,63,70,149,362.0,"50,000+",93463,True,0.00,19000000.0,#open,NaN,NaN,Mature 17+,False,True,False,17.0,1026.0


In [19]:
count = 0
hashMap = {}
for i in range(df.shape[0]):
    temp = list(df.loc[i, "CATEGORY"].split(","))
    num_of_categories = len(temp)
    if (num_of_categories not in hashMap):
        hashMap[num_of_categories] = 1
    else:
        hashMap[num_of_categories] +=1
# iterate hashmap
for k, v in hashMap.items():
    print(f"{v} apps have {k} categories")

17454 apps have 1 categories
21 apps have 2 categories
1 apps have 4 categories


### Modify `CATEGORY` column

In [20]:
for i in range(df.shape[0]):
    temp = list(df.loc[i, "CATEGORY"].split(","))
    num_of_categories = len(temp)
    if (num_of_categories == 4):
        app_with_four_cat = pd.DataFrame(df.loc[i, ])
        break
app_name_with_four_cat = app_with_four_cat[i]["APP_NAME"]
app_name_with_four_cat

'HD Video Screen Mirroring'

We will handle the apps that belongs to more than 1 category (Pick one of the category of the 2, we drop the one with 4 category since its stats doesnt matter anyways)

In [21]:
count = 0
sanity_check_df = []
for i in range(df.shape[0]):
    temp = list(df.loc[i, "CATEGORY"].split(","))
    num_of_categories = len(temp)
    if (num_of_categories == 2):
        df.loc[i, "CATEGORY"] = str(temp[0])
        sanity_check_df.append(df.loc[i, ])
        count +=1
print("There are", count, "apps that belongs to more than one category.")
sanity_check_df = pd.DataFrame(sanity_check_df)
sanity_check_df.shape

There are 21 apps that belongs to more than one category.


(21, 24)

In [22]:
#Drop the entry that belongs to 4 categories
df = df[df["APP_NAME"] != app_name_with_four_cat]
df.reset_index(inplace=True, drop=True)
#Remove the brackets around `CATEGORY` row values
for i in range(df.shape[0]):
    category = df.loc[i, "CATEGORY"]
    if (category[-1] != "]"):
        df.loc[i, "CATEGORY"] = category[2:-1]
    else:
        df.loc[i, "CATEGORY"] = category[2:-2]
df.head()

,APP_NAME,RATING,CATEGORY,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,REVIEW_COUNT,INSTALLS,MAX_INSTALLS,FREE,PRICE,SIZE,DEVELOPER,DEVELOPER_ADDRESS,COUNTRY,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,DAYS_SINCE_UPDATE,DAYS_SINCE_RELEASED
0,"""Ghost Voice Catcher"" AUTO EVP",3.933333,Lifestyle,68,12,1,6,7,42,24.0,"1,000+",3608,False,5.99,1100000.0,BIG BEARD Studios,114 1/2 Ash st. Nashua NH 03060,Usa,Everyone,False,False,False,1820.0,3482.0
1,"""OXXO""",4.740000,Puzzle,975,23,0,15,94,843,54.0,"10,000+",21680,False,0.99,81000000.0,Hamster On Coke Games,ul Jugosłowiańska 37F/10\n73-110 Stargard Szcz...,NaN,Everyone,False,False,False,0.0,945.0
2,#DRIVE,4.434152,Racing,229679,9352,4740,15249,47804,152534,6473.0,"5,000,000+",9119244,True,0.00,132000000.0,Pixel Perfect Dude S.A.,ul. Polna 28/6 lok. 4\n62-800 Kalisz\nPolska,NaN,Everyone,True,True,False,4.0,1084.0
3,#SelfCare,4.463476,Role Playing,14932,865,336,1088,1353,11290,1501.0,"1,000,000+",1209692,True,0.00,83000000.0,TRU LUV Inc.,"186-720 King Street West\nToronto, Ontario, Ca...",NaN,Teen,False,True,False,952.0,1275.0
4,#open Polyamorous + ENM Dating,2.550000,Dating,708,263,163,63,70,149,362.0,"50,000+",93463,True,0.00,19000000.0,#open,NaN,NaN,Mature 17+,False,True,False,17.0,1026.0


<a id = 'memory optimization'></a>
# Memory Optimization

In [23]:
df.dtypes

APP_NAME                object
RATING                 float64
CATEGORY                object
RATING_COUNT             int64
1_STAR_RATINGS           int64
2_STAR_RATINGS           int64
3_STAR_RATINGS           int64
4_STAR_RATINGS           int64
5_STAR_RATINGS           int64
REVIEW_COUNT           float64
INSTALLS                object
MAX_INSTALLS             int64
FREE                      bool
PRICE                  float64
SIZE                   float64
DEVELOPER               object
DEVELOPER_ADDRESS       object
COUNTRY                 object
CONTENT_RATING          object
AD_SUPPORTED              bool
IN_APP_PURCHASES          bool
EDITORS_CHOICE            bool
DAYS_SINCE_UPDATE      float64
DAYS_SINCE_RELEASED    float64
dtype: object

In [24]:
def to_float32(df, *args):
    for col_name in args:
        df[col_name] = df[col_name].astype("float32")
        
def to_int32(df, *args):
    for col_name in args:
        df[col_name] = df[col_name].astype("int32") 

def to_int16(df, *args):
    for col_name in args:
        df[col_name] = df[col_name].astype("int16") 

In [25]:
to_float32(df, ["RATING", "REVIEW_COUNT", "PRICE"])
to_int32(df, ["1_STAR_RATINGS", "2_STAR_RATINGS", "3_STAR_RATINGS", "4_STAR_RATINGS", "5_STAR_RATINGS"])

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17475 entries, 0 to 17474
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   APP_NAME             17475 non-null  object 
 1   RATING               17475 non-null  float32
 2   CATEGORY             17475 non-null  object 
 3   RATING_COUNT         17475 non-null  int64  
 4   1_STAR_RATINGS       17475 non-null  int32  
 5   2_STAR_RATINGS       17475 non-null  int32  
 6   3_STAR_RATINGS       17475 non-null  int32  
 7   4_STAR_RATINGS       17475 non-null  int32  
 8   5_STAR_RATINGS       17475 non-null  int32  
 9   REVIEW_COUNT         17475 non-null  float32
 10  INSTALLS             17475 non-null  object 
 11  MAX_INSTALLS         17475 non-null  int64  
 12  FREE                 17475 non-null  bool   
 13  PRICE                17475 non-null  float32
 14  SIZE                 17475 non-null  float64
 15  DEVELOPER            17475 non-null 

In order to improve our productivity, we have converted some fields to a smaller version of the variable type. Doing so, helps us remove unnecessary storage and enable us to accelerate our code runs.

In [27]:
# exporting the cleaned data into a csv file
file_path = "./datasets/google_app_scrap_cleaned.csv"
df.to_csv(path_or_buf=file_path, index=False);